In [1]:
#DiamondCalc.py created 2/27/2023 by BWM and EGM
#program to calculate heat load and current in diamond devices

import numpy as np
import os
import matplotlib.pyplot as plt
import math
cwd = os.getcwd()

##############################################
#Get elemental data in and properly formed
##############################################
carbon = np.loadtxt((cwd + "/sourceData/carbon.txt"), dtype = float,comments ="#",)
pt = np.loadtxt((cwd + "/sourceData/pt.txt"), dtype = float,comments ="#",)
al = np.loadtxt((cwd + "/sourceData/al.txt"), dtype = float,comments ="#",)
###############################################
# Constants
###############################################
densC = 3.15 #g/cm^3
densPt = 21.4 #g/cm^3
densAl = 2.7 #g/cm^3
enConv = 1.60218e-19 #eV per Joule
coulPerElec =  6.28e18  #elec per s for amp
micro_cm = 1e-4 # microns to cm conversion
nano_cm = 1e-7 # nm to cm conversion
ToMev= 1e-6
freeCarrier = 13.2 #energy to make electorn whole pairs in diamond
################################################


def MAdevice_perf_al (diamon_thick, metal_thick,energy, flux):
    bias = 10 #bias on dimaond
    coefAl = np.interp((energy*ToMev), al[:,0], al[:,2])
    coefC = np.interp((energy*ToMev), carbon[:,0], carbon[:,2])
    transAl = (math.exp(-coefAl*metal_thick*nano_cm*densPt))
    ptLayerAbs = (flux * (1-transAl))
    flux_to_diamond =flux-ptLayerAbs
    transC = (math.exp(-coefC*diamon_thick*micro_cm*densC))
    cLayerAbs = flux_to_diamond * (1-transC)
    current = (cLayerAbs * energy * enConv/ (freeCarrier))
    absHeat = (ptLayerAbs * 2 + cLayerAbs) * energy * enConv
    jouleHeat = current * bias
    heatLoad = absHeat + jouleHeat 
    absAlall = (1-transAl) * 2
    absC = 1-transC
    return current , heatLoad, absAlall, absC

def MAdevice_perf_pt (diamon_thick, metal_thick,energy, flux):
    bias = 10 #bias on dimaond
    coefC = np.interp((energy*ToMev), carbon[:,0], carbon[:,2])
    coefPt = np.interp((energy*ToMev), pt[:,0], pt[:,2])
    transPt = (math.exp(-coefPt*metal_thick*nano_cm*densPt))
    ptLayerAbs = (flux * (1-transPt))
    flux_to_diamond =flux-ptLayerAbs
    transC = (math.exp(-coefC*diamon_thick*micro_cm*densC))
    cLayerAbs = flux_to_diamond * (1-transC)
    current = (cLayerAbs * energy * enConv/ (freeCarrier))
    absHeat = (ptLayerAbs * 2 + cLayerAbs) * energy * enConv
    jouleHeat = current * bias
    totalHeat = absHeat + jouleHeat 
    absPtall = (1-transPt) * 2
    absC = 1-transC
    return current , totalHeat, absPtall, absC, absHeat




   



## How to use
MAdevice_perf_pt returns a tuple. 
- 0 = current
- 1 = heatload 
- 2 = absorbtion in both metal layers
- 3 = absorbtion in diamond

In [19]:
ID4 = {2.75:1e15, 6.0:1e15, 27.0:1e13}
ID8 = {4.7:1e15, 7.5:1e15, 25:1e13, 26.05:1e13}
ID9 = {6:1e15, 30:1e13}

In [58]:
BL = {"4ID":ID4, "8ID":ID8, "9ID":ID9}

for line in BL:
    print("{} device performance".format(line))

    for eneg in BL[line]:
        eneV = eneg*1000
        flux = BL[line][eneg]
        MADevice = MAdevice_perf_pt(45,30,eneV,flux)
        totPow = (MADevice[2]+MADevice[3]) * eneV * enConv
        
        print ("for Energy {} keV with {:1.2e} ph/s \nExpected Currnet {:1.2e} A, Expected Heat {:1.2e} W, Absorbed energy in Device {:1.2e} W\n".format(eneg, flux,MADevice[0], MADevice[1], MADevice[4]))
        



4ID device performance
for Energy 2.75 keV with 1.00e+15 ph/s 
Expected Currnet 2.48e-02 A, Expected Heat 7.02e-01 W, Absorbed energy in Device 4.54e-01 W

for Energy 6.0 keV with 1.00e+15 ph/s 
Expected Currnet 9.85e-03 A, Expected Heat 2.77e-01 W, Absorbed energy in Device 1.79e-01 W

for Energy 27.0 keV with 1.00e+13 ph/s 
Expected Currnet 5.25e-06 A, Expected Heat 3.15e-04 W, Absorbed energy in Device 2.63e-04 W

8ID device performance
for Energy 4.7 keV with 1.00e+15 ph/s 
Expected Currnet 1.58e-02 A, Expected Heat 4.38e-01 W, Absorbed energy in Device 2.80e-01 W

for Energy 7.5 keV with 1.00e+15 ph/s 
Expected Currnet 7.09e-03 A, Expected Heat 2.02e-01 W, Absorbed energy in Device 1.31e-01 W

for Energy 25 keV with 1.00e+13 ph/s 
Expected Currnet 6.21e-06 A, Expected Heat 3.65e-04 W, Absorbed energy in Device 3.03e-04 W

for Energy 26.05 keV with 1.00e+13 ph/s 
Expected Currnet 5.74e-06 A, Expected Heat 3.40e-04 W, Absorbed energy in Device 2.83e-04 W

9ID device performance
for 

In [3]:
list2 = {4.8:1e25,7:1e15,30:1e13, 9:1e15}
for eneg in list2:
        eneV = eneg*1000
        flux = list2[eneg]
        MADevice = MAdevice_perf_pt(45,30,eneV,flux)
        totPow = (MADevice[2]+MADevice[3]) * eneV * enConv
        
        print ("for Energy {} keV with {:1.2e} ph/s \nExpected Currnet {:1.2e} A, Expected Heat {:1.2e} W, Absorbed energy in Device {:1.2e} W\n".format(eneg, flux,MADevice[0], MADevice[1], MADevice[4]))

for Energy 4.8 keV with 1.00e+25 ph/s 
Expected Currnet 1.51e+08 A, Expected Heat 4.20e+09 W, Absorbed energy in Device 2.69e+09 W

for Energy 7 keV with 1.00e+15 ph/s 
Expected Currnet 8.29e-03 A, Expected Heat 2.34e-01 W, Absorbed energy in Device 1.52e-01 W

for Energy 30 keV with 1.00e+13 ph/s 
Expected Currnet 3.41e-06 A, Expected Heat 2.19e-04 W, Absorbed energy in Device 1.85e-04 W

for Energy 9 keV with 1.00e+15 ph/s 
Expected Currnet 4.74e-03 A, Expected Heat 1.38e-01 W, Absorbed energy in Device 9.02e-02 W



In [9]:

eneV = 3000
flux = 2e15
MADevice = MAdevice_perf_pt(45,30,eneV,flux)
totPow = (MADevice[2]+MADevice[3]) * eneV * enConv

print ("for Energy {} keV with {:1.2e} ph/s \nExpected Currnet {:1.2e} A, Expected Heat {:1.2e} W, Absorbed energy in Device {:1.2e} W\n".format(eneV, flux,MADevice[0], MADevice[1], MADevice[4]))

for Energy 3000 keV with 2.00e+15 ph/s 
Expected Currnet 4.63e-02 A, Expected Heat 1.30e+00 W, Absorbed energy in Device 8.35e-01 W

